# Model Testing

This notebook introduces basic concepts in testing simulation models. There are two parts organized as 15 min of group lecture and 45 minutes of break out.

1. Testing basics
  - Lecture
      - Motivtion
      - What is a test and why test
      - Setting up tests in a Jupyter Notebook
  - Breakout: Set up tests for virus example
1. Writing tests
  - Lecture
    - Smoke test
    - One off test
    - Boundary test
    - Relative tests
  - Breakout: Write one of each type of tests for the virus model


In [1]:
import sys
sys.path

['/home/ubuntu/Model-Testing',
 '/home/ubuntu/Model-Testing/mt/lib/python3.6/site-packages/teUtils-0.85-py3.6.egg',
 '',
 '/home/ubuntu/Model-Testing/mt/lib/python3.6/site-packages',
 '/home/ubuntu/Model-Testing/mt/lib/python3.6/site-packages/lmfit-1.0.1-py3.6.egg',
 '/home/ubuntu/Model-Testing/mt/lib/python3.6/site-packages/tabulate-0.8.7-py3.6.egg',
 '/home/ubuntu/Model-Testing/mt/lib/python3.6/site-packages/asteval-0.9.18-py3.6.egg',
 '/home/ubuntu/Model-Testing/mt/lib/python3.6/site-packages/future-0.18.2-py3.6.egg',
 '/home/ubuntu/Model-Testing/mt/lib/python3.6/site-packages/IPython/extensions',
 '/home/ubuntu/Model-Testing/mt/lib/python3.6/site-packages/tesbml',
 '/home/ubuntu/miniconda3/lib/python36.zip',
 '/home/ubuntu/miniconda3/lib/python3.6',
 '/home/ubuntu/miniconda3/lib/python3.6/lib-dynload',
 '/home/ubuntu/miniconda3/lib/python3.6/site-packages',
 '/home/ubuntu/Model-Testing/mt/lib/python3.6/site-packages/IPython/extensions',
 '/home/ubuntu/.ipython']

In [2]:
import tellurium as te
from teUtils.named_timeseries import NamedTimeseries, TIME

ModuleNotFoundError: No module named 'uncertainties'

## Testing Basics

### Motivation and Background

Testing is the process by which you exercise your code to determine if it performs as expected. The code you are testing is referred to as the code under test.

There are two parts to writing tests.
1. invoking the code under test so that it is exercised in a particular way;
1. evaluating the results of executing code under test to determine if it behaved as expected.

The collection of tests performed are referred to as the test cases. The fraction of the code under test that is executed as a result of running the test cases is referred to as test coverage.

### Testing in a Jupyter Notebook

In [2]:
# Common data container
class CommonData():
    # Comtainer for common data
    pass
global common

In [3]:
# Common codes used by tests
def setUp():
    global common
    # Initialize common data
    common = CommonData()
    # Run the simulation
    model = '''
    model example1
      S1 -> S2; k1*S1
      S1 = 10
      S2 = 0
      k1 = 0.1
    end
    '''
    # Collect results of simulation run
    common.rr = te.loada(model)
    common.data = common.rr.simulate()

In [4]:
# Tests
def test1():
    global common
    setUp()
    assert(len(common.data) > 0)
def test2():
    setUp()
    pass

In [5]:
# Test runner
for test in [test1, test2]:
    test()
print("OK.")

OK.


### Testing A More Sophisticated Model

In [6]:
ANTIMONY_MODEL = """ 
# Reactions   
    J1: S1 -> S2; k1*S1
    J2: S2 -> S3; k2*S2
    J3: S3 -> S4; k3*S3
    J4: S4 -> S5; k4*S4
    J5: S5 -> S6; k5*S5;
# Species initializations     
    S1 = 10;
    k1 = 1; k2 = 2; k3 = 3; k4 = 4; k5 = 5;
    S1 = 0; S2 = 0; S3 = 0; S4 = 0; S5 = 0; S6 = 0;
"""

In [14]:
# Run the simulation to be tested
def setUp():
    global common
    # Initialize common data
    common = CommonData()
    # Run the simulation
    model = '''
        # Reactions   
        J1: S1 -> S2; k1*S1
        J2: S2 -> S3; k2*S2
        J3: S3 -> S4; k3*S3
        J4: S4 -> S5; k4*S4
        J5: S5 -> S6; k5*S5;
        # Species initializations     
        S1 = 10;
        k1 = 1; k2 = 2; k3 = 3; k4 = 4; k5 = 5;
        S1 = 10; S2 = 0; S3 = 0; S4 = 0; S5 = 0; S6 = 0;
        '''
    # Collect results of simulation run
    common.model = model
    common.rr = te.loada(model)
    common.data = common.rr.simulate()

In [15]:
# Test runner
for test in [test1, test2]:
    test()
print("OK.")

OK.


### Breakout
You will be writing tests for ANTIMONY_MODEL
1. Create a new Jupyter Notebook
1. Create cells for:
   1. Common data container
   1. Simulation runner for the virus example
   1. Test that checks that data are returned from the simulation.
1. Create a new test that verifies the columns in the simulation results.
   1. Test that the beginning value of S1 is near 10 and its ending value of S1 is near 0.
   1. Test that the beginning value of S6 is near 0 and its ending value is near 10.

## Writing Kinetics Tests

Test cases can be of several types. Below are listed some common classifications of test cases.
- Smoke test. This is an invocation of the code under test to see if there is an unexpected exception. It's useful as a starting point, but this doesn't tell you anything about the correctness of the results of a computation.
- One-shot test. In this case, you call the code under test with arguments for which you know the expected result.
- Edge test. The code under test is invoked with arguments that should cause an exception, and you evaluate if the expected exception occurrs.
- Pattern test - Based on your knowledge of the calculation (not implementation) of the code under test, you construct a suite of test cases for which the results are known or there are known patterns in these results that are used to evaluate the results returned.

Another principle of testing is to limit what is done in a single test case. Generally, a test case should focus on one use of one function. Sometimes, this is a challenge since the function being tested may call other functions that you are testing. This means that bugs in the called functions may cause failures in the tests of the calling functions. Often, you sort this out by knowing the structure of the code and focusing first on failures in lower level tests. In other situations, you may use more advanced techniques called mocking. A discussion of mocking is beyond the scope of this course